In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/students-drop-out-prediction/train.csv
/kaggle/input/students-drop-out-prediction/test.csv


In [2]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, f1_score
from sklearn.neural_network import MLPClassifier

import seaborn as sns
sns.set(rc={'figure.figsize':(20, 20)})

In [3]:
train_df = pd.read_csv("../input/students-drop-out-prediction/train.csv")
test_df = pd.read_csv("../input/students-drop-out-prediction/test.csv")

In [4]:
all_col_names = list(set(train_df.columns) - set(["label", "id"])) 
binary_col_names = ["v_1", "v_26", "v_11", "v_14", "v_30", "v_28", "v_9", "v_27"]
non_binary_col_names = list(set(all_col_names) - set(binary_col_names))

In [5]:
transformer = ColumnTransformer(
    [
        ("drop_id", "drop", ["id"]),
        ("scale_non_binary", StandardScaler(), non_binary_col_names)
    ],
    remainder="passthrough"
)
transform_train_df = transformer.fit_transform(train_df)
feature_names = [name.split("__")[1] for name in transformer.get_feature_names_out()]
transform_train_df = pd.DataFrame(data=transform_train_df, columns=feature_names)
transform_train_df = transform_train_df.astype(dict([(col, "int") for col in binary_col_names]))

In [6]:
transform_train_df1 = transform_train_df.copy()
y_train_com = transform_train_df1.pop("label")
X_train_com = transform_train_df1
X_train, X_valid, y_train, y_valid = train_test_split(X_train_com, y_train_com, test_size=0.3, random_state=1)

# untuned MLP
model = MLPClassifier(random_state=1)
model.fit(X_train, y_train)
preds = model.predict(X_valid)
print(classification_report(y_valid, preds))

param_grid = {
    "hidden_layer_sizes": [(10), (30), (10, 10)],
    "max_iter": [500, 1000],
    "solver": ["sgd", "adam"],
    "activation": ["relu", "sigmoid"],
    "alpha": [0.01, 0.1],
}
model = MLPClassifier(random_state=1, learning_rate="adaptive")
search = GridSearchCV(model, param_grid, scoring="f1_macro", refit=True, cv=3, verbose=1)
search.fit(X_train_com, y_train_com)
print(search.best_params_)
print(search.best_score_)

{'activation': 'relu', 'alpha': 0.1, 'hidden_layer_sizes': 30, 'max_iter': 1000, 'solver': 'sgd'}
0.6585231231872934

model = MLPClassifier(
    random_state=1,
    learning_rate="adaptive",
    hidden_layer_sizes=40,
    max_iter=1000,
    solver="sgd",
    activation="relu",
    alpha=0.1
)
model.fit(X_train, y_train)
preds = model.predict(X_valid)
print(classification_report(y_valid, preds))

In [7]:
model = MLPClassifier(
    random_state=1,
    learning_rate="adaptive",
    hidden_layer_sizes=40,
    max_iter=1000,
    solver="sgd",
    activation="relu",
    alpha=0.1
)
model.fit(X_train_com, y_train_com)

# We do not need 'id' column
# Scale non binary features
test_df = pd.read_csv("../input/students-drop-out-prediction/test.csv")
transform_test_data = transformer.fit_transform(test_df)
feature_names = [name.split("__")[1] for name in transformer.get_feature_names_out()]
transform_test_df = pd.DataFrame(data=transform_test_data, columns=feature_names)
transform_test_df = transform_test_df.astype(dict([(col, "int") for col in binary_col_names]))
preds = [int(i) for i in model.predict(transform_test_df)]
submission = pd.DataFrame({"id": test_df["id"].to_numpy(), "label": preds})
submission.to_csv("submission.csv", index=False)

/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
